# Validate a Model Against an Expected Value

In [1]:
import types
import schematics


class BaseModel(schematics.Model):
    def _repr_info(self):
        try:
            return self.name
        except AttributeError:
            return None
    
    @property
    def expected(self):
        try:
            return self._expected
        except AttributeError:
            self._expected = {}
            return self._expected
    
    @expected.setter
    def expected(self, value):
        self._expected = value
            
    def expect(self, expected=None, **kwargs):
        if expected is not None:
            self.expected = expected
        self.expected.update(kwargs)
        return self
    
    def check_field(self, field_name, value):
        try:
            expected = self.expected[field_name]
        except KeyError:
            return
        if value != expected:
            raise schematics.exceptions.ValidationError(f"Expected={expected}, actual={value}")
            
    def validate(self, expected=None, **kwargs):
        if expected is not None:
            self.expected = expected
        super().validate(**kwargs)


class EnvironmentResponseMetadata(BaseModel):
    name = schematics.types.StringType(required=True)
    description = schematics.types.StringType()

    def validate_name(self, _, value):
        self.check_field("name", value)
            
    def validate_description(self, _, value):
        self.check_field("description", value)

In [2]:
e = EnvironmentResponseMetadata({"name": "stagging", "description": "Stagging environment"})
try:
    e.expect(name="foo", description="bar")
    e.validate()
except schematics.exceptions.DataError as error:
    print(f"Error: {error}")

Error: {"name": ["Expected=foo, actual=stagging"], "description": ["Expected=bar, actual=Stagging environment"]}


In [3]:
# We can combine expect and validate together into 1 line if they are short enough
e = EnvironmentResponseMetadata({"name": "stagging", "description": "Stagging environment"})
try:
    e.expect(name="foo", description="bar").validate()
except schematics.exceptions.DataError as error:
    print(f"Error: {error}")

Error: {"name": ["Expected=foo, actual=stagging"], "description": ["Expected=bar, actual=Stagging environment"]}


In [4]:
# We can combine expect and validate together into 1 line if they are short enough
e = EnvironmentResponseMetadata({"name": "stagging", "description": "Stagging environment"})
try:
    e.expect({"name": "test", "description": "A test environment"}).validate()
except schematics.exceptions.DataError as error:
    print(f"Error: {error}")

Error: {"name": ["Expected=test, actual=stagging"], "description": ["Expected=A test environment, actual=Stagging environment"]}


In [5]:
# Use validate with expected values
e = EnvironmentResponseMetadata({"name": "stagging", "description": "Stagging environment"})
try:
    e.validate(expected={"name": "test", "description": "A test environment"})
except schematics.exceptions.DataError as error:
    print(f"Error: {error}")

Error: {"name": ["Expected=test, actual=stagging"], "description": ["Expected=A test environment, actual=Stagging environment"]}
